In [17]:
import pandas as pd

# ------------------------------------------------------------
# 1) Daten einlesen
#    - sep=";" wegen Semikolon-Trennung
#    - decimal="," wegen Dezimalkomma in numerischen Spalten
#    - dtype=str, damit beim Einlesen nichts "kaputt" typisiert wird
# ------------------------------------------------------------
path = r"data\original_bridge_statistic_germany.csv"  # <-- anpassen (raw string!)

df = pd.read_csv(
    path,
    sep=";",
    decimal=",",
    dtype=str,
    encoding="utf-8"
)

# ------------------------------------------------------------
# 2) Relevante Spalten numerisch machen (fehlende Werte -> NaN)
# ------------------------------------------------------------
df["Jahr letzte Hauptprüfung"] = pd.to_numeric(df["Jahr letzte Hauptprüfung"], errors="coerce")
df["Substanzkennzahl"] = pd.to_numeric(df["Substanzkennzahl"], errors="coerce")
df["Zustandsnote"] = pd.to_numeric(df["Zustandsnote"], errors="coerce")

# ------------------------------------------------------------
# 3) Auf Thüringen & Sachsen-Anhalt filtern
# ------------------------------------------------------------
target_states = ["Thueringen", "Sachsen-Anhalt", "Thüringen"]  # robust gegen beide Schreibweisen
df_two = df[df["Bundeslandname"].isin(target_states)].copy()

# ------------------------------------------------------------
# 4) Durchschnitt "Jahr letzte Hauptprüfung" pro Bundesland
# ------------------------------------------------------------
avg_last_main_inspection = (
    df_two.groupby("Bundeslandname", dropna=False)["Jahr letzte Hauptprüfung"]
    .mean()
    .round(2)
    .reset_index(name="Ø Jahr letzte Hauptprüfung")
)

print("Durchschnitt Jahr der letzten Hauptprüfung (TH / ST):")
print(avg_last_main_inspection)

# ------------------------------------------------------------
# 5) Durchschnitt "Substanzkennzahl" pro Bundesland
# ------------------------------------------------------------
avg_substanzkennzahl = (
    df_two.groupby("Bundeslandname", dropna=False)["Substanzkennzahl"]
    .mean()
    .round(3)
    .reset_index(name="Ø Substanzkennzahl")
)

print("\nDurchschnitt Substanzkennzahl (TH / ST):")
print(avg_substanzkennzahl)

# ------------------------------------------------------------
# 6) Durchschnitt "Zustandsnote" pro Bundesland
# ------------------------------------------------------------
avg_zustandsnote = (
    df_two.groupby("Bundeslandname", dropna=False)["Zustandsnote"]
    .mean()
    .round(3)
    .reset_index(name="Ø Zustandsnote")
)

print("\nDurchschnitt Zustandsnote (TH / ST):")
print(avg_zustandsnote)

# ------------------------------------------------------------
# 7) (Optional) Wenn du die zwei Bundesländer als einzelne Zahlen brauchst:
# ------------------------------------------------------------
th_values = df_two[df_two["Bundeslandname"].isin(["Thueringen", "Thüringen"])]
st_values = df_two[df_two["Bundeslandname"] == "Sachsen-Anhalt"]

print("\nEinzeln:")
print("Ø Jahr letzte Hauptprüfung Thüringen:", round(th_values["Jahr letzte Hauptprüfung"].mean(), 2))
print("Ø Jahr letzte Hauptprüfung Sachsen-Anhalt:", round(st_values["Jahr letzte Hauptprüfung"].mean(), 2))
print("Ø Substanzkennzahl Thüringen:", round(th_values["Substanzkennzahl"].mean(), 3))
print("Ø Substanzkennzahl Sachsen-Anhalt:", round(st_values["Substanzkennzahl"].mean(), 3))
print("Ø Zustandsnote Thüringen:", round(th_values["Zustandsnote"].mean(), 3))
print("Ø Zustandsnote Sachsen-Anhalt:", round(st_values["Zustandsnote"].mean(), 3))


Durchschnitt Jahr der letzten Hauptprüfung (TH / ST):
   Bundeslandname  Ø Jahr letzte Hauptprüfung
0  Sachsen-Anhalt                     2020.62
1      Thueringen                     2020.77

Durchschnitt Substanzkennzahl (TH / ST):
   Bundeslandname  Ø Substanzkennzahl
0  Sachsen-Anhalt               2.046
1      Thueringen               1.919

Durchschnitt Zustandsnote (TH / ST):
   Bundeslandname  Ø Zustandsnote
0  Sachsen-Anhalt           2.138
1      Thueringen           1.924

Einzeln:
Ø Jahr letzte Hauptprüfung Thüringen: 2020.77
Ø Jahr letzte Hauptprüfung Sachsen-Anhalt: 2020.62
Ø Substanzkennzahl Thüringen: 1.919
Ø Substanzkennzahl Sachsen-Anhalt: 2.046
Ø Zustandsnote Thüringen: 1.924
Ø Zustandsnote Sachsen-Anhalt: 2.138


In [18]:
import pandas as pd

# ------------------------------------------------------------
# 1) Daten einlesen
# ------------------------------------------------------------
path = r"data\original_bridge_statistic_germany.csv"  # <-- anpassen
df = pd.read_csv(path, sep=";", decimal=",", dtype=str, encoding="utf-8")

# ------------------------------------------------------------
# 2) Numerische Spalten
# ------------------------------------------------------------
df["Jahr letzte Hauptprüfung"] = pd.to_numeric(df["Jahr letzte Hauptprüfung"], errors="coerce")
df["Substanzkennzahl"] = pd.to_numeric(df["Substanzkennzahl"], errors="coerce")
df["Zustandsnote"] = pd.to_numeric(df["Zustandsnote"], errors="coerce")

# ------------------------------------------------------------
# 3) Filter: Thüringen & Sachsen-Anhalt
# ------------------------------------------------------------
target_states = ["Thueringen", "Thüringen", "Sachsen-Anhalt"]
df_two = df[df["Bundeslandname"].isin(target_states)].copy()

# ------------------------------------------------------------
# 4) Durchschnitt pro Kreis (innerhalb der Bundesländer)
#    -> liefert pro (Bundeslandname, Kreis) die Mittelwerte
# ------------------------------------------------------------
avg_by_kreis = (
    df_two.groupby(["Bundeslandname", "Kreis"], dropna=False)
    .agg(
        n=("ObjectID", "count"),
        avg_last_main_inspection=("Jahr letzte Hauptprüfung", "mean"),
        avg_substanzkennzahl=("Substanzkennzahl", "mean"),
        avg_zustandsnote=("Zustandsnote", "mean"),
    )
    .reset_index()
)

# Rundung (Jahr meist ganzzahlig sinnvoll, aber ich lasse 2 Stellen zur Sicherheit)
avg_by_kreis["avg_last_main_inspection"] = avg_by_kreis["avg_last_main_inspection"].round(2)
avg_by_kreis["avg_substanzkennzahl"] = avg_by_kreis["avg_substanzkennzahl"].round(3)
avg_by_kreis["avg_zustandsnote"] = avg_by_kreis["avg_zustandsnote"].round(3)

# Sortierung: erst Bundesland, dann Kreis
avg_by_kreis = avg_by_kreis.sort_values(["Bundeslandname", "Kreis"])

print(avg_by_kreis)

# Optional: als CSV speichern
# avg_by_kreis.to_csv("avg_by_kreis_th_st.csv", sep=";", index=False, encoding="utf-8")


    Bundeslandname                          Kreis    n  \
0   Sachsen-Anhalt         Altmarkkreis Salzwedel   41   
1   Sachsen-Anhalt              Anhalt-Bitterfeld   91   
2   Sachsen-Anhalt                Burgenlandkreis  170   
3   Sachsen-Anhalt                          Börde  218   
4   Sachsen-Anhalt          Dessau-Rosslau, Stadt   41   
5   Sachsen-Anhalt           Halle (Saale), Stadt   24   
6   Sachsen-Anhalt                           Harz  169   
7   Sachsen-Anhalt                Jerichower Land   92   
8   Sachsen-Anhalt    Magdeburg, Landeshauptstadt   20   
9   Sachsen-Anhalt               Mansfeld-Südharz  150   
10  Sachsen-Anhalt                     Saalekreis  224   
11  Sachsen-Anhalt                  Salzlandkreis  159   
12  Sachsen-Anhalt                        Stendal   63   
13  Sachsen-Anhalt                     Wittenberg  117   
14  Sachsen-Anhalt                            NaN    2   
15      Thueringen                EISENACH, STADT   54   
16      Thueri

In [19]:
import pandas as pd
import plotly.express as px

# ------------------------------------------------------------
# 1) Daten einlesen
# ------------------------------------------------------------
path = r"data\original_bridge_statistic_germany.csv"  # <-- anpassen
df = pd.read_csv(path, sep=";", decimal=",", dtype=str, encoding="utf-8")

# numerische Spalten
num_cols = ["Jahr letzte Hauptprüfung", "Substanzkennzahl", "Zustandsnote"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Filter: Thüringen & Sachsen-Anhalt (robust gegen Schreibweise)
target_states = ["Thueringen", "Thüringen", "Sachsen-Anhalt"]
df_two = df[df["Bundeslandname"].isin(target_states)].copy()

# ------------------------------------------------------------
# 2) Aggregation: Durchschnitt pro Kreis innerhalb Bundesland
# ------------------------------------------------------------
avg_by_kreis = (
    df_two.groupby(["Bundeslandname", "Kreis"], dropna=False)
    .agg(
        n=("ObjectID", "count"),
        avg_year=("Jahr letzte Hauptprüfung", "mean"),
        avg_substanz=("Substanzkennzahl", "mean"),
        avg_zustand=("Zustandsnote", "mean"),
    )
    .reset_index()
)

# Rundung für Anzeige
avg_by_kreis["avg_year"] = avg_by_kreis["avg_year"].round(2)
avg_by_kreis["avg_substanz"] = avg_by_kreis["avg_substanz"].round(3)
avg_by_kreis["avg_zustand"] = avg_by_kreis["avg_zustand"].round(3)

# Hilfslabel für Hover
avg_by_kreis["hover"] = (
    "<b>" + avg_by_kreis["Kreis"].astype(str) + "</b><br>"
    "Bundesland: " + avg_by_kreis["Bundeslandname"].astype(str) + "<br>"
    "n Brücken: " + avg_by_kreis["n"].astype(str) + "<br>"
    "Ø Jahr letzte HP: " + avg_by_kreis["avg_year"].astype(str) + "<br>"
    "Ø Zustandsnote: " + avg_by_kreis["avg_zustand"].astype(str) + "<br>"
    "Ø Substanzkennzahl: " + avg_by_kreis["avg_substanz"].astype(str)
)

# ------------------------------------------------------------
# Plot 1: Scatter (Kreis-Mittelwerte)
#   x: Jahr letzte Hauptprüfung (Ø)
#   y: Zustandsnote (Ø)
#   color: Substanzkennzahl (Ø)
#   size: Anzahl Brücken
# ------------------------------------------------------------
fig_scatter = px.scatter(
    avg_by_kreis,
    x="avg_year",
    y="avg_zustand",
    color="avg_substanz",
    size="n",
    facet_col="Bundeslandname",
    hover_name="Kreis",
    hover_data={"avg_year": True, "avg_zustand": True, "avg_substanz": True, "n": True},
    title="Kreis-Durchschnitte: Zustandsnote vs. Jahr letzte Hauptprüfung (Größe=n, Farbe=Substanzkennzahl)",
)

fig_scatter.update_layout(
    template="plotly_white",
    xaxis_title="Ø Jahr der letzten Hauptprüfung",
    yaxis_title="Ø Zustandsnote",
)
fig_scatter.show()

# ------------------------------------------------------------
# Plot 2: Ranking-Balken (schlechteste Zustandsnoten)
#   Top N Kreise pro Bundesland (höhere Zustandsnote = schlechter)
# ------------------------------------------------------------
TOP_N = 15

rank_df = (
    avg_by_kreis.dropna(subset=["avg_zustand"])
    .sort_values(["Bundeslandname", "avg_zustand"], ascending=[True, False])
    .groupby("Bundeslandname", as_index=False)
    .head(TOP_N)
    .copy()
)

fig_bar = px.bar(
    rank_df,
    x="avg_zustand",
    y="Kreis",
    facet_col="Bundeslandname",
    orientation="h",
    hover_data={"avg_year": True, "avg_substanz": True, "n": True},
    title=f"Top {TOP_N} Kreise mit der schlechtesten Ø Zustandsnote (je Bundesland)",
)

fig_bar.update_layout(
    template="plotly_white",
    xaxis_title="Ø Zustandsnote (höher = schlechter)",
    yaxis_title="Kreis",
)
# damit die schlechtesten oben stehen
fig_bar.for_each_yaxis(lambda ax: ax.update(categoryorder="total ascending"))
fig_bar.show()

# ------------------------------------------------------------
# Plot 3: Verteilungen auf Einzelbrücken-Ebene (nicht aggregiert)
#   Violin/Box nach Bundesland für die 3 Kennzahlen
# ------------------------------------------------------------
# Zustandsnote
fig_violin_z = px.violin(
    df_two.dropna(subset=["Zustandsnote"]),
    x="Bundeslandname",
    y="Zustandsnote",
    box=True,
    points="all",
    title="Verteilung Zustandsnote (Einzelbrücken) nach Bundesland",
)
fig_violin_z.update_layout(template="plotly_white", xaxis_title="Bundesland", yaxis_title="Zustandsnote")
fig_violin_z.show()

# Substanzkennzahl
fig_violin_s = px.violin(
    df_two.dropna(subset=["Substanzkennzahl"]),
    x="Bundeslandname",
    y="Substanzkennzahl",
    box=True,
    points="all",
    title="Verteilung Substanzkennzahl (Einzelbrücken) nach Bundesland",
)
fig_violin_s.update_layout(template="plotly_white", xaxis_title="Bundesland", yaxis_title="Substanzkennzahl")
fig_violin_s.show()

# Jahr letzte Hauptprüfung
fig_violin_y = px.violin(
    df_two.dropna(subset=["Jahr letzte Hauptprüfung"]),
    x="Bundeslandname",
    y="Jahr letzte Hauptprüfung",
    box=True,
    points="all",
    title="Verteilung Jahr letzte Hauptprüfung (Einzelbrücken) nach Bundesland",
)
fig_violin_y.update_layout(template="plotly_white", xaxis_title="Bundesland", yaxis_title="Jahr letzte Hauptprüfung")
fig_violin_y.show()
